<a href="https://colab.research.google.com/github/guillerZeva/Bot-Agencia-de-Viajes-Telegram/blob/main/RTravelBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot
!pip install emoji
!apt install chromium-chromedriver
!pip install selenium

#Clases Usuario y ticket

In [1]:
class Ticket:  
  vuelo = None
  nu_asientos = 0


class User:
  tickets = []
  nombre = ''
  apellido = ''
 

#Clases para el Vuelo

In [2]:
class Disponible:
  def __init__(self, lista_destino):
    self.lista_destino = lista_destino 


class Vuelo :
  fecha_final = 'None' 
  
  def __init__(self):
    pais_origen=''
    iata_origen = ''
    origen = ''
    aeropuerto_origen = ''
    pais_destino=''
    iata_destino = ''
    destino = ''
    aeropuerto_destino = ''
    aerolinea = ''
    hora_salida = ''
    etapas = ''
    hora_llegada = ''
    costo = ''
    fecha_inicial = ''

class Vuelos_opciones:
  def __init__(self):
    lista_vuelos = []

#Emojize

In [3]:
from emoji import emojize

emoji_sonrisa = emojize(":smile:", use_aliases=True)
emoji_estrellas = emojize(":sparkles:", use_aliases=True)
emoji_mundo = emojize(":earth_americas:", use_aliases=True)
emoji_avion = emojize(":airplane:", use_aliases=True)
emoji_buscar = emojize(":mag_right:", use_aliases=True)
emoji_cod = emojize(":man_technologist:", use_aliases=True)
emoji_dia_azul = emojize(":small_blue_diamond:", use_aliases=True)
emoji_estrella = emojize(":star:", use_aliases=True)
emoji_ticket = emojize(":ticket:", use_aliases=True)
emoji_fecha = emojize(":date:", use_aliases=True)
emoji_reloj = emojize(":hourglass:", use_aliases=True)
emoji_asiento = emojize(":seat:", use_aliases=True)
emoji_bangbang = emojize(":bangbang:", use_aliases=True)
emoji_help = emojize(":grey_question:", use_aliases=True)	
emoji_dolar = emojize(":heavy_dollar_sign:", use_aliases=True)	
emoji_carita = emojize(":slightly_frowning_face:", use_aliases=True)

buscar_por = f'Para buscar el lugar: {emojize(":world_map:", use_aliases=True)} {emojize(":mag:", use_aliases=True)}\n{emojize(":keyboard:", use_aliases=True)} Puede ingresar en el teclado {emojize(":keyboard:", use_aliases=True)}\n@TravelSafely_bot iata \n @TravelSafely_bot ciudad\n @TravelSafely_bot airport'

#Scraping Despegar

In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import  By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(chrome_options=options)

def buscar_vuelos_ida(origen, destino, fecha,tipo):

  url ='https://www.despegar.com.ec/shop/flights/results/oneway/'+origen+'/'+destino+'/'+fecha+'/1/0/0/NA/NA/NA/NA?from=SB&di=1-0'

  driver.get(url)
  time.sleep(1.5)
  c = 1
  lista = []
  while c < 5:
    texto = driver.find_elements_by_css_selector('#clusters > span:nth-child('+str(c)+') > span > cluster > div > div')
    if len(texto) ==0:
      break
    lista.append(texto)
    c += 1 

  list_vuelos = []
  for filto in lista:
    contenido = filto[0].text.replace(' \n', '').split('\n')
    c = 0
    viaje = []
    fil = ['meses sin intereses','¡Puedes sumar puntos' ,'Vuelo super flexible: podrás cambiar la fecha sin pagar multa.','Conocer más','IDA','Equipaje','¡Puedes sumar','Seleccionar','Ver opciones',
          'Llega más rápido, ¡Vuela directo!','Vuelo flexible: podrás cambiar la fecha sin pagar multa.','Ver bancos y tarjetas','Arma el vuelo a tu medida',
          'Ver bancos y tarjetas',  '  Únicamente en un pago', 'Precio final','Suma equipaje, selección de asiento y más', 'Clase']
    for i in contenido:
      if not i in fil:
        if not fil[0] in i:
          if not fil[1] in i:
            viaje.append(i)
      c += 1

      
    vuelo = Vuelo()
    vuelo.fecha_inicial = viaje[0]
    vuelo.iata_origen = viaje[1]
    vuelo.origen = viaje[2]
    vuelo.aeropuerto_origen = Buscar_json ('Aeropuerto','IATA',vuelo.iata_origen)
    vuelo.pais_origen = Buscar_json_pais(vuelo.iata_origen )
    vuelo.iata_destino = viaje[3]
    vuelo.destino = viaje[4]
    vuelo.aeropuerto_destino = Buscar_json('Aeropuerto','IATA',vuelo.iata_destino)
    vuelo.pais_destino = Buscar_json_pais(vuelo.iata_destino )
    vuelo.aerolinea = viaje[5]
    vuelo.hora_salida = viaje[6]
    vuelo.etapas = viaje[7]
    vuelo.hora_llegada = viaje[8]
    c = viaje[-1].replace('USD','').replace('.','')
    costo = int(c)    

    if tipo == 'ida_vuelta':
      aux = costo * 2
      vuelo.costo = str(aux)
    else:
      vuelo.costo = c  

    list_vuelos.append(vuelo)

  return list_vuelos


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


#Manejo de json

In [5]:
import json
from emoji import emojize

def emoji_pais(pais):
  with open('paises.json', encoding='utf-8') as contenido:
    paises = json.load(contenido)
    valor ='' 
    for p in paises:
          if p['pais'] == pais:
            emoji = p['emoji']
            valor = emoji
  return valor



def Buscar_json_pais(iataCiudad):
  with open('IATA.json') as contenido:
    iatas = json.load(contenido)
    valor ='' 
    for pais in iatas:
      for i in iatas[pais]:

        if i['IATA'] == iataCiudad or iataCiudad in i['Ciudad'] or i['Ciudad'] in iataCiudad or iataCiudad == i['Aeropuerto']:
          valor = pais

  return valor


def lista_json(tipo):
  with open('IATA.json') as contenido:
    iatas = json.load(contenido)
    valor = [] 
    for pais in iatas:
      for i in iatas[pais]:
        if tipo == 'IATA':
          valor.append(i['IATA'])
        elif tipo == 'Ciudad':
          valor.append(i['Ciudad'])
        elif tipo == 'Aeropuerto':
          valor.append(i['Aeropuerto'])

  return valor

  
def Buscar_json(tipo,de,busca):

  retorno = lista_json(tipo)
  desde = lista_json(de)
  valor ='' 
  for b in zip (retorno,desde):
    if b[1] == busca:
      valor = b[0]

  return valor

def paises_viaje(viajes):
  paises = []
  for v in viajes:
    viaje = v.destino + " " + emojize(":"+emoji_pais(Buscar_json_pais(v.iata_destino))+":", use_aliases=True)
    paises.append(viaje)
    
  return paises

#Scraping auxiliar 

In [6]:
from bs4 import BeautifulSoup as bs
import requests
import random

def contiene(vuelos,destino):
  no = True
  for i in vuelos:
    if i.destino == destino:
      no = False
      break
  return no

def buscar_viaje2(ciudad):
  ciudades = lista_json('Ciudad')
  vuelos = set ()
  
  for v in range(7):
    destino = random.choice(ciudades)

    if contiene(vuelos,destino) :
      viaje = Vuelo()
      viaje.pais_origen = Buscar_json_pais(ciudad) 
      viaje.pais_destino = Buscar_json_pais(destino) 
      viaje.origen = ciudad
      viaje.iata_origen = Buscar_json('IATA','Ciudad',ciudad)
      viaje.destino = destino
      viaje.iata_destino = Buscar_json('IATA','Ciudad',destino) 
      vuelos.add(viaje)
      
  list_viajes = list(vuelos)
  return list_viajes


def buscar_viaje(ciudad):

  url ='https://www.latam.com/es-ec/vuelos-desde-'+ ciudad.lower()
  res = requests.get(url)
  soup = bs(res.content)
  lugares = soup.find_all('div', class_="card-body")
  vuelos = set ()

  for v in lugares:
    destino = v.find('span', class_='fare-atom-origin-destination-destination-city')
    fechas = v.find_all('p', class_='fare-atom-trip-leg-date')
    iataD = v.find('span', class_='fare-atom-origin-destination-destination-airport')

    if ( destino != None and contiene(vuelos,destino.get_text())):
      iata = iataD.get_text().replace(' ','').replace('(','').replace(')','')
      
      viaje = Vuelo()
      
      viaje.pais_origen = Buscar_json_pais(ciudad) 
      viaje.pais_destino = Buscar_json_pais(destino.get_text()) 
      viaje.origen = ciudad
      viaje.iata_origen = Buscar_json('IATA','Ciudad',ciudad)
      viaje.destino = destino.get_text()
      viaje.iata_destino = iata 

      vuelos.add(viaje)

  list_viajes = list(vuelos)
  return list_viajes

def enlistar_vuelos(vuelos):
  lista = []
  for v in vuelos:
    texto = v.iata_origen + ' -> ' + v.iata_destino +f' {emoji_avion}:'+v.aerolinea+f' {emoji_dolar}:'+v.costo 
    lista.append(texto)

  return lista

#**Calendar**

In [7]:
from telegram import InlineKeyboardButton, InlineKeyboardMarkup,ReplyKeyboardRemove
import datetime
import calendar

def create_callback_data(action,year=0,month=0,day=0):
    return ";".join([action,str(year),str(month),str(day)])

def separate_callback_data(data):
    lis = data.split(";")
    if lis[0] != 'DAY' and lis[0] != 'IGNORE' and lis[0] != 'PREV-MONTH' and lis[0] != 'NEXT-MONTH':
      lis = ['IGNORE',2020,1,0]

    return lis


def create_calendar(year=None,month=None):

    now = datetime.datetime.now()
    if year == None: year = now.year
    if month == None: month = now.month
    data_ignore = create_callback_data("IGNORE", year, month, 0)
    keyboard = []
    #First row - Month and Year
    row=[]
    row.append(InlineKeyboardButton(calendar.month_name[month]+" "+str(year)+f" {emoji_fecha}",callback_data=data_ignore))
    keyboard.append(row)
    #Second row - Week Days
    row=[]
    for day in ["Mo","Tu","We","Th","Fr","Sa","Su"]:
        row.append(InlineKeyboardButton(day,callback_data=data_ignore))
    keyboard.append(row)

    my_calendar = calendar.monthcalendar(year, month)
    for week in my_calendar:
        row=[]
        for day in week:
            if(day==0):
                row.append(InlineKeyboardButton(" ",callback_data=data_ignore))
            else:
                row.append(InlineKeyboardButton(str(day),callback_data=create_callback_data("DAY",year,month,day)))
        keyboard.append(row)

    return InlineKeyboardMarkup(keyboard)


def process_calendar_selection(bot,update):

    ret_data = (False,None)
    query = update.callback_query

    (action,year,month,day) = separate_callback_data(query.data)
    curr = datetime.datetime(int(year), int(month), 1)
    if action == "IGNORE":
        bot.answer_callback_query(callback_query_id= query.id)
    elif action == "DAY":
        bot.edit_message_text(text=query.message.text,
            chat_id=query.message.chat_id,
            message_id=query.message.message_id
            )
        ret_data = True,datetime.datetime(int(year),int(month),int(day))
    else:
        bot.answer_callback_query(callback_query_id= query.id,text="Algo salió mal!")
        # UNKNOWN
    return ret_data

#Botones 


In [8]:
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, ReplyKeyboardRemove,KeyboardButton
from geopy.geocoders import Nominatim

def build_menu(buttons,n_cols,header_buttons=None,footer_buttons=None):
  menu = [buttons[i:i + n_cols] for i in range(0, len(buttons), n_cols)]
  if header_buttons:
    menu.insert(0, header_buttons)
  if footer_buttons:
    menu.append(footer_buttons)
  return menu

def mostrarBotones(lista,columns,callback):
  button_list = []
  for each in lista:

    button_list.append(InlineKeyboardButton(each, callback_data = each))
    

  reply_markup=InlineKeyboardMarkup(build_menu(button_list,n_cols=columns)) 
  return reply_markup


def botones_menu():
  vd = 'Vuelos Disponible'+' '+str(emoji_avion)
  b = 'Buscar'+' '+str(emoji_buscar)

    
  botones_menu = [[InlineKeyboardButton(vd, callback_data = 'Vuelos_Disponible' )],
                  [InlineKeyboardButton(b, callback_data = 'buscar')]]

  reply_markup = InlineKeyboardMarkup(botones_menu)
  return reply_markup

def escoger_origen( ):
  main_menu_keyboard = [[KeyboardButton(text= 'Ubicacion Actual',request_location = True)],
                          [KeyboardButton('Otro lugar')]]
  reply_kb_markup = ReplyKeyboardMarkup(main_menu_keyboard,
                                                   resize_keyboard=True,
                                                   one_time_keyboard=True)
  return reply_kb_markup

def coger_location(update,context):

  punto = str(update.message.location.latitude)+','+str(update.message.location.longitude)

  geolocalizador = Nominatim()
  direccion = geolocalizador.reverse(punto)
  aux = direccion.address.split(', ')
    
  return aux[3]

def texto_escoger_ticket(lugar,vuelo):
  emoji_ida = emojize(":ticket:", use_aliases=True)
  ti = emoji_ida + ' ' + 'Ticket ida'
  tiv = emoji_ida + ' ' + 'Ticket ida y vuelta'
      
  origen = emojize(":"+emoji_pais(vuelo.pais_origen)+":", use_aliases=True) +' '+vuelo.pais_origen
  destino = emojize(":"+emoji_pais(vuelo.pais_destino)+":", use_aliases=True) +' '+vuelo.pais_destino 

  text = origen +': '+ vuelo.origen +' > > '+ destino +': ' +lugar

  return text

def botones_tickets_ida_vuelta():

  ti = emoji_ticket + ' ' + 'Ticket ida'
  tiv = emoji_ticket + ' ' + 'Ticket ida y vuelta'

  botones_menu = [[InlineKeyboardButton(ti, callback_data = 'ticket_ida' )],
                  [InlineKeyboardButton(tiv, callback_data = 'ticket_ida_vuelta')]]

  reply_markup = InlineKeyboardMarkup(botones_menu)
  return reply_markup

def copiaVuelo(vu):
    vuelo = Vuelo()    
    vuelo.fecha_inicial =  vu.fecha_inicial  
    vuelo.fecha_final = vu.fecha_final 
    vuelo.iata_origen =  vu.iata_origen 
    vuelo.origen =     vu.origen      
    vuelo.aeropuerto_origen =  vu.aeropuerto_origen  
    vuelo.pais_origen =   vu.pais_origen        
    vuelo.iata_destino =     vu.iata_destino      
    vuelo.destino =        vu.destino          
    vuelo.aeropuerto_destino =   vu.aeropuerto_destino 
    vuelo.pais_destino =    vu.pais_destino    
    vuelo.aerolinea =   vu.aerolinea        
    vuelo.hora_salida =   vu.hora_salida       
    vuelo.etapas =   vu.etapas           
    vuelo.hora_llegada =  vu.hora_llegada      
    vuelo.costo =  vu.costo
    return vuelo           

def realizar_busqueda(vuelos_opciones,viaje,tipo):
    co = 0
    vali = True
    while co < 3 and vali: 
      vuelos_opciones.lista_vuelos = buscar_vuelos_ida(viaje.iata_origen,  viaje.iata_destino, viaje.fecha_inicial,tipo)
      vali = False if len(vuelos_opciones.lista_vuelos) != 0 else True
      co += 1


def presentando_ticket(update, context, ticket, usuario):
    vuelo = ticket.vuelo
    update.message.reply_text(f'{emojize(":ticket:", use_aliases=True)} TICKET {emojize(":ticket:", use_aliases=True)}\n\n'+
                              f'Usuario {emojize(":robot:", use_aliases=True)}: {usuario.nombre}\t\t{usuario.apellido}\n\n'+
                              f'{emojize(":airplane:", use_aliases=True)} VUELO {emojize(":airplane:", use_aliases=True)}\n'+
                              f'Origen: {vuelo.origen}{vuelo.iata_origen}--> Destino: {vuelo.destino} {vuelo.iata_destino}\n'+
                              f'Fecha {emojize(":date:", use_aliases=True)}: {vuelo.fecha_inicial} -- {vuelo.fecha_final}\n '+
                              f'Aeropuertos {emojize(":european_post_office:", use_aliases=True)}:\n'+
                              f'Salida: {vuelo.aeropuerto_origen}\n'+
                              f'Llegada: {vuelo.aeropuerto_destino}\n'+
                              f'Hora Salida {emojize(":alarm_clock:", use_aliases=True)}: {vuelo.hora_salida}\n'+
                              f'Asientos {emojize(":seat:", use_aliases=True)}: {ticket.nu_asientos}\n'
                              f'Etapas: {emojize(":triangular_flag_on_post:", use_aliases=True)}: {vuelo.etapas}\n'+
                              f'Costo {emojize(":dollar:", use_aliases=True)}: ${vuelo.costo}'
                              ) 



#TRAVELBOT

In [ ]:
import logging
from uuid import uuid4
from telegram import InlineQueryResultArticle, ParseMode, InputTextMessageContent, Update, ReplyKeyboardMarkup
from telegram.ext import Updater, InlineQueryHandler, CommandHandler, CallbackContext, CallbackQueryHandler
from telegram.ext import ConversationHandler, MessageHandler,Filters
from emoji import emojize
import copy
INPUT,OTRO_LUGAR,TICKET,COMPRAR,ASIENTO,VALIDAR_NUMERO_ASIENTO= range(6)
ORIGEN,IN_DESTINO = range(2)
FECHAIDA = range(7,8)
FECHAIDA_V, FECHAVUELTA_V = range(7,9)
END = ConversationHandler.END

updater = Updater("TOKEN")

dispatcher = updater.dispatcher

usuario = User()
lista = []
disponible = Disponible(lista)
viaje = Vuelo()
vuelos_opciones = Vuelos_opciones()
ticket = Ticket()
lista_inline_query_iata = lista_json('IATA')
lista_inline_query_ciudad = lista_json('Ciudad')
lista_inline_query_Aero = lista_json('Aeropuerto')


def start(update,context):
  print(update.message.chat)
  usuario.nombre = update.message.chat.first_name
  usuario.apellido = update.message.chat.last_name

  update.message.reply_text(f'¡Hola, {usuario.nombre} {emoji_sonrisa}!  \n'+
                              f'{emoji_estrellas}¡Bienvenido a Travel Bot!{emoji_estrellas}\n'+
                              '\nTravel bot te ayudará a buscar y reservar un vuelo'+ 
                              f' desde donde estés hacia cualquier parte del mundo {emoji_avion}{emoji_mundo}.'+
                              f'\nUsar /menu \nPara saber mas información sobre los comandos /help')

def help (update,context):
  update.message.reply_text(f'{emoji_dia_azul} Comandos {emoji_dia_azul} \n'+
                            '/menu Muestra opciones de reserva.\n'+
                            '/tickets Lista con todos los tickets.\n'
                            '/help presenta lista de comandos.\n'+
                            '/cancel cancela cualquier proceso.\n'+
                            '/start Mensaje de bienvenida.')
  
def menu(update, context) :
  text_menu = f'{emoji_estrellas} Menú de navegación {emoji_estrellas}\n'
  reply_markup = botones_menu()
  context.bot.send_message(chat_id=update.message.chat_id, text=text_menu ,reply_markup=reply_markup)


def cancel(update, context) :
  lista = []
  disponible = Disponible(lista)
  viaje = Vuelo()
  vuelos_opciones = Vuelos_opciones()
  ticket = Ticket()
  main_menu_keyboard = [[KeyboardButton(text= '/start')],
                          [KeyboardButton('/Tickets')]]
  reply_kb_markup = ReplyKeyboardMarkup(main_menu_keyboard,
                                                  resize_keyboard=True,
                                                  one_time_keyboard=True)

  update.message.reply_text('Proceso finalizado.',reply_markup=reply_kb_markup)
  all_hand = dispatcher.handlers
  for dict_group in all_hand:
      for handler in all_hand[dict_group]:
          if isinstance(handler, ConversationHandler):
              print(handler)
              handler.update_state(ConversationHandler.END, handler._get_key(update))

  return ConversationHandler.END
  
#conversacion Vuelos disponibles ----------------------------------------------------------------

def lugar_Vuelos_Disponible(update,context):
  query = update.callback_query
  query.answer()

  reply_markup = escoger_origen()
  context.bot.send_message(chat_id=query.message.chat_id,
                          text=f'{emoji_dia_azul} Escoger el lugar de origen {emoji_dia_azul}',
                          reply_markup=reply_markup) 

  query.edit_message_text( f'{emoji_estrella} Viajes Disponibles {emoji_estrella}\n\n'+
                          f'Esta opción le permite reservar un vuelo desde un lugar de origen hacia cualquier parte. {emoji_mundo}\n')

  return INPUT

def viaje_disponible_otroLugar(update,context):
  texto = 'OTRO LUGAR: \n' + buscar_por
  update.message.reply_text( texto )

  return OTRO_LUGAR

def viaje_disponible_input_lugar(update,context):
    ciudad = ''
    c = update.message.text
    print(c)
    if len(c) == 3:
      ciudad = Buscar_json('Ciudad','IATA',c)
    elif len(c) > 15:
      ciudad = Buscar_json('Ciudad','Aeropuerto',c)
    else:
      ciudad =  c


    disponible.lista_destino = buscar_viaje(ciudad) if len(buscar_viaje(ciudad)) != 0 else buscar_viaje2(ciudad)  
    list_of_cities = paises_viaje(disponible.lista_destino)
    reply_markup = mostrarBotones(list_of_cities,1,'viajes')
    update.message.reply_text(text=f'Viajes Disponibles desde: {ciudad}', reply_markup=reply_markup)
 
    return TICKET


def viaje_disponible_location(update,context):
  loc = coger_location(update,context)
  disponible.lista_destino = buscar_viaje(loc) if len(buscar_viaje(loc)) != 0 else buscar_viaje2(loc) 
  print(disponible.lista_destino[0].origen)

  list_of_cities = paises_viaje(disponible.lista_destino)
  reply_markup = mostrarBotones(list_of_cities,1,'viajes')
  update.message.reply_text(text=f'Viajes Disponibles desde {loc}: ', reply_markup=reply_markup)
  return TICKET

def ticket(update,context):
  query = update.callback_query
  query.answer()

  list_of_cities = paises_viaje(disponible.lista_destino)

  for lugar in zip(list_of_cities, disponible.lista_destino):
    if update.callback_query.data == lugar[0]:
      
      viaje.origen = lugar[1].origen
      viaje.iata_origen = lugar[1].iata_origen
      viaje.destino = lugar[1].destino
      viaje.iata_destino  = lugar[1].iata_destino 

      text = texto_escoger_ticket(lugar[0],lugar[1])

      reply_markup = botones_tickets_ida_vuelta()

      query.edit_message_text(text=text , reply_markup=reply_markup)

  return COMPRAR

def ticket_ida(update,context):
  query = update.callback_query
  query.answer()
  query.edit_message_text(text=f'Escoger Fecha para la ida {emoji_fecha}:' )
  context.bot.send_message(chat_id=update.callback_query.from_user.id,
                           text="Por favor escoger una fecha de ida: ",
                          reply_markup=create_calendar())
  
  return FECHAIDA  

def escoger_vuelo_ida(update, context):
  selected,date = process_calendar_selection(context.bot, update)
  if selected:
    viaje.fecha_inicial = date.strftime("%Y-%m-%d")

    texto = f'La fecha de partida es: {date.strftime("%d/%m/%Y")}\nEspere unos segundos {emoji_reloj}'

    context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text=texto,
                        reply_markup=ReplyKeyboardRemove())
        
    selected,date = (False,None)
    realizar_busqueda(vuelos_opciones, viaje, 'ida')
    print(len(vuelos_opciones.lista_vuelos))
    if len(vuelos_opciones.lista_vuelos) != 0:
      reply_markup = mostrarBotones(enlistar_vuelos(vuelos_opciones.lista_vuelos),1,'vuelos')
      context.bot.send_message(chat_id=update.callback_query.from_user.id,
                              text=f'Escoger un vuelo:',
                              reply_markup=reply_markup)
    else:
      text = f'No se ha encontrado vuelos hacia {viaje.destino} {emoji_carita}'
      context.bot.send_message(chat_id=update.callback_query.from_user.id,
                              text=text)
      cancel(update, context)

    return END 
  
def ticket_ida_vuelta(update,context):
  query = update.callback_query
  query.answer()
  query.edit_message_text(text=f'Escoger Fecha (Ida y vuelta): {emoji_fecha}' )
  context.bot.send_message(chat_id=update.callback_query.from_user.id,
                           text="Por favor escoger una fecha: ",
                          reply_markup=create_calendar())
  
  return FECHAIDA_V

def escoger_fecha_ida_IV(update, context):
    selected,date = process_calendar_selection(context.bot, update)
    if selected:
      viaje.fecha_inicial = date.strftime("%Y-%m-%d")

      context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text="La fecha de partida es: %s" % (date.strftime("%d/%m/%Y")))
      texto = f'{emoji_dia_azul} Regreso {emojize(emoji_dia_azul)}'

      context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text=texto,
                        reply_markup = create_calendar())
        
    return FECHAVUELTA_V

def escoger_fecha_vuelta_IV(update, context):
    selected,date = process_calendar_selection(context.bot, update)
    if selected:
        viaje.fecha_final = date.strftime("%Y-%m-%d")

        texto = f'La fecha de regreso es: {date.strftime("%d/%m/%Y")}\nEspere unos segundos {emoji_reloj}'

        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text=texto,
                        reply_markup=ReplyKeyboardRemove())
        
        selected,date = (False,None)
        
        
        realizar_busqueda(vuelos_opciones, viaje, 'ida_vuelta')
        print(len(vuelos_opciones.lista_vuelos))
        reply_markup = mostrarBotones(enlistar_vuelos(vuelos_opciones.lista_vuelos),1,'vuelos')
        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                             text=f'Vuelos de (ida vuelta):',
                             reply_markup=reply_markup)

    return END

def cant_asientos (update, context):
    query = update.callback_query
    query.answer()

    for vuelo in zip(enlistar_vuelos(vuelos_opciones.lista_vuelos), vuelos_opciones.lista_vuelos):
      if update.callback_query.data == vuelo[0]:
        ticket.vuelo = copiaVuelo(vuelo[1])

        print('vuelos: ',id(vuelo[1]),id(ticket.vuelo))
        ticket.vuelo.fecha_final = viaje.fecha_final
        viaje.aeropuerto_origen = vuelo[1].aeropuerto_origen
        query.edit_message_text(f'Numero de asientos {emoji_asiento}: ')
    
    return VALIDAR_NUMERO_ASIENTO

def valida_numero_asiento (update, context):
  try:
    numero = int(update.message.text)
    nueTi = Ticket()
    nueTi.nu_asientos = numero
    nueTi.vuelo = copiaVuelo(ticket.vuelo)
    usuario.tickets.append(nueTi)
    print('tickets: ',id(ticket),id(usuario.tickets))
    update.message.reply_text(f'Bien{emojize(":ok_hand:", use_aliases=True)}')

    presentando_ticket(update, context, nueTi, usuario)

    return cancel(update, context)

  except ValueError:
    update.message.reply_text(f'{emoji_bangbang} ha sucedido un error {emoji_bangbang}'+
                              '\nUse /cancel para intentar de nuevo.')
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------BUSCAR -----------------------------------------------------------------------------
def buscar (update, context):
    query = update.callback_query
    query.answer()
    query.edit_message_text('Buscar:')
    reply_markup = escoger_origen()
    context.bot.send_message(chat_id=query.message.chat_id,
                          text=f'{emoji_dia_azul} Escoger el lugar de origen {emoji_dia_azul}',
                          reply_markup=reply_markup) 
    return ORIGEN

def buscar_location(update, context):
    loc = coger_location(update,context)
    
    viaje.pais_origen = Buscar_json_pais(loc)
    viaje.origen = loc
    viaje.iata_origen = Buscar_json('IATA','Ciudad',loc)
    viaje.aeropuerto_origen = Buscar_json('Aeropuerto','IATA',viaje.iata_origen)
    update.message.reply_text(f'{viaje.origen} c:')
    texto = 'DESTINO: \n' +  buscar_por
    update.message.reply_text(texto)

    return IN_DESTINO 
 
def buscar_otroLugar(update, context): 
    
    texto = 'ORIGEN: \n' +  buscar_por

    update.message.reply_text(texto)

    return OTRO_LUGAR

def tran_iata(text):
    iata = ''
    c = text

    if len(c) > 15:
      iata = Buscar_json('IATA','Aeropuerto',c)
    elif len(c) != 3:
      iata = Buscar_json('IATA','Ciudad',c)
    else:
      iata =  c
    return iata 

def buscar_input_lugar(update, context): 

    iata = tran_iata(update.message.text)
    print('--------------------------------------<AUIE>')
    viaje.pais_origen = Buscar_json_pais(iata)
    viaje.origen = Buscar_json('Ciudad','IATA',iata)
    viaje.iata_origen = iata
    viaje.aeropuerto_origen = Buscar_json('Aeropuerto','IATA',iata)
    update.message.reply_text(f'{viaje.origen} c:')
    texto = 'DESTINO: \n' +  buscar_por
    update.message.reply_text( texto )

    return IN_DESTINO

def buscar_input_destion(update, context): 

    iata = tran_iata(update.message.text)
    viaje.pais_destino = Buscar_json_pais(iata)
    viaje.destino = Buscar_json('Ciudad','IATA',iata)
    viaje.iata_destino = iata
    viaje.aeropuerto_destino = Buscar_json('Aeropuerto','IATA',iata)
    update.message.reply_text(f'{viaje.destino} c:')
   
    text = texto_escoger_ticket(viaje.destino, viaje)

    reply_markup = botones_tickets_ida_vuelta()

    update.message.reply_text(text=text , reply_markup=reply_markup)
    
    return COMPRAR

def buscar_ticket(update, context): 

    text = texto_escoger_ticket(viaje.destino, viaje)
    reply_markup = botones_tickets_ida_vuelta()
    update.message.reply_text(text=text , reply_markup=reply_markup)

    return COMPRAR


#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def buscar_porIATA(update, context): 
    query = update.inline_query.query
    
    results = []
    for m in lista_inline_query_iata:
      results.append(InlineQueryResultArticle(
                    id=uuid4(),
                    title=m,
                    input_message_content=InputTextMessageContent(m)))
    
    update.inline_query.answer(results) 

def buscar_porCiudad(update, context): 
    query = update.inline_query.query

    results = []
    for m in lista_inline_query_ciudad:
      results.append(InlineQueryResultArticle(
                    id=uuid4(),
                    title=m,
                    input_message_content=InputTextMessageContent(m)))
    
    update.inline_query.answer(results) 

def buscar_porAero(update, context): 
    query = update.inline_query.query

    results = []
    for m in lista_inline_query_Aero:
      results.append(InlineQueryResultArticle(
                    id=uuid4(),
                    title=m,
                    input_message_content=InputTextMessageContent(m)))
    
    update.inline_query.answer(results) 

def mis_tickets(update, context):

    update.message.reply_text(f'{emojize(":pushpin:", use_aliases=True)} Mis Tickets {emojize(":pushpin:", use_aliases=True)}')
    for ti in usuario.tickets:
      print(ti.vuelo.destino)
      presentando_ticket(update, context,ti,usuario)

    update.message.reply_text(f'{emojize(":small_blue_diamond:", use_aliases=True)} use /cancel para finalizar esta reserva y volver a empezar {emojize(":small_blue_diamond:", use_aliases=True)}' )



def main() -> None:

  dispatcher.add_handler(CommandHandler("start", start))

  dispatcher.add_handler(CommandHandler("menu", menu))

  dispatcher.add_handler(CommandHandler("help", help))
  
  dispatcher.add_handler(CommandHandler("tickets", mis_tickets))

  dispatcher.add_handler(CommandHandler("cancel", cancel))

  #ticket ida y vuelta----
  ticket_ida_vuelta_handler = ConversationHandler(
  entry_points=[CallbackQueryHandler(callback= ticket_ida_vuelta, pattern='ticket_ida_vuelta')],
  states={
      FECHAIDA_V:      [CallbackQueryHandler(escoger_fecha_ida_IV)],
      FECHAVUELTA_V:   [CallbackQueryHandler(escoger_fecha_vuelta_IV)]

  },
  fallbacks=[CommandHandler("cancel", cancel)],
  map_to_parent={
            END: ASIENTO
        }
  )

  #Ticket ida----------
  ticket_ida_handler = ConversationHandler(
  entry_points=[CallbackQueryHandler(callback= ticket_ida, pattern='ticket_ida')],
  states={
      FECHAIDA: [CallbackQueryHandler(escoger_vuelo_ida)]
  },
  fallbacks=[CommandHandler("cancel", cancel)],
  map_to_parent={
            END: ASIENTO
        }
  )

  #conversacion Vuelos disponibles -------------------------------------------
  vuelos_handler = ConversationHandler(
  entry_points=[CallbackQueryHandler(callback= lugar_Vuelos_Disponible, pattern='Vuelos_Disponible')],
  states={
      INPUT:      [MessageHandler(Filters.text, viaje_disponible_otroLugar),MessageHandler(Filters.location, viaje_disponible_location)],
      OTRO_LUGAR: [MessageHandler(Filters.text, viaje_disponible_input_lugar)],
      TICKET:     [CallbackQueryHandler(callback= ticket)],
      COMPRAR:    [ticket_ida_vuelta_handler,
                    ticket_ida_handler],
      ASIENTO:    [CallbackQueryHandler(callback= cant_asientos)],
      VALIDAR_NUMERO_ASIENTO: [MessageHandler(Filters.text, valida_numero_asiento)]
  },
  fallbacks=[]
  )
  #conversacion Busca -------------------------------------------------------------------------------
  buscar_handler = ConversationHandler(
  entry_points=[CallbackQueryHandler(callback= buscar, pattern='buscar')],
  states={
      ORIGEN: [MessageHandler(Filters.text, buscar_otroLugar),MessageHandler(Filters.location, buscar_location)],
      OTRO_LUGAR: [MessageHandler(Filters.text, buscar_input_lugar)],
      IN_DESTINO: [MessageHandler(Filters.text, buscar_input_destion)],
      TICKET: [MessageHandler(Filters.text, buscar_ticket)],
      COMPRAR:[ticket_ida_vuelta_handler,
            ticket_ida_handler],
      ASIENTO: [CallbackQueryHandler(callback= cant_asientos)],
      VALIDAR_NUMERO_ASIENTO: [MessageHandler(Filters.text, valida_numero_asiento)]
  },
  fallbacks=[CommandHandler("cancel", cancel)]
  )

  dispatcher.add_handler(vuelos_handler,2)

  dispatcher.add_handler(buscar_handler,3)

  dispatcher.add_handler(InlineQueryHandler(buscar_porIATA, pattern='iata'))
  
  dispatcher.add_handler(InlineQueryHandler(buscar_porCiudad, pattern='ciudad'))
  
  dispatcher.add_handler(InlineQueryHandler(buscar_porAero, pattern='airport'))

  updater.start_polling()

  updater.idle()

if __name__ == '__main__':
  
  main()

/usr/local/lib/python3.7/dist-packages/telegram/ext/conversationhandler.py:280: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  "If 'per_message=False', 'CallbackQueryHandler' will not be "


El Oro
0
